In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import pandas as pd
import numpy as np
import re
from src import etl as etl

## Importamos la información

In [3]:
path = 's3://inai-summerofdata/parquet/inai.parquet'
df = pd.read_parquet(path, engine='pyarrow')
pd.options.display.max_columns = None

In [4]:
df.head()

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012/01/01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
1,1857500002012,2012/01/01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
2,0001800000712,2012/01/01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Inexistencia de la información solicitada,EL ASUNTO SE TURNÓ A LA DIRECCIÓN GENERAL DE R...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0,None
3,0000600003712,2012/01/01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0,INIDOS SECRETARIA DE HACIENDA Y CREDITO PUBLI...
4,0001500002812,2012/01/01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Entrega de información en medio electrónico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0,None


## Estandarización de nulos 

In [5]:
df = df.replace(r'^\s*$', np.nan, regex=True)

## Validación del tipo de las columnas

En esta sección nos aseguraremos de que cada columna tenga el tipo correcto.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559012 entries, 0 to 1559011
Data columns (total 23 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   FOLIO                        1559012 non-null  object 
 1   FECHASOLICITUD               1559012 non-null  object 
 2   DEPENDENCIA                  1559012 non-null  object 
 3   ESTATUS                      1559012 non-null  object 
 4   MEDIOENTRADA                 1559012 non-null  object 
 5   TIPOSOLICITUD                1559012 non-null  object 
 6   DESCRIPCIONSOLICITUD         1559000 non-null  object 
 7   OTROSDATOS                   617311 non-null   object 
 8   ARCHIVOADJUNTOSOLICITUD      180445 non-null   object 
 9   MEDIOENTREGA                 1559011 non-null  object 
 10  FECHALIMITE                  1392031 non-null  object 
 11  RESPUESTA                    1559012 non-null  object 
 12  TEXTORESPUESTA               1457769 non-n

**fechas**

In [7]:
df['FECHASOLICITUD'] = pd.to_datetime(df['FECHASOLICITUD'])

In [8]:
df['FECHALIMITE'] = pd.to_datetime(df['FECHALIMITE'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 19-04-22 00:00:00

Obtuvimos un error al tratar de convertir a formato de fecha la columna "FECHALIMITE", el error nos explica que existen fechas fuera de rango.

Primero obtendremos las fechas 2019/04/22, considerando que es la fecha que nos esta dando problemas segun el log del error.

In [9]:
filtro = df['FECHALIMITE'] == '2019/04/22'
aux = df[filtro]
folios_delete = aux['FOLIO'].unique()
len(aux)

1005

Ahora obtendremos las fechas que contienen 19/04/22.

In [10]:
filtro = df['FECHALIMITE'].str.contains('19/04/22', na=False, regex=True)
aux1 = df[filtro]
len(aux1)

1009

Ahora obtenemos las diferencias entre los 2 dataframes auxiliares para identificar los registros con inconsistencias.

In [11]:
aux2 = aux1[~aux1.FOLIO.isin(folios_delete)]
aux2

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
480809,0001000025519,2019-02-05 18:27:36,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,Proporcione el contrato -y sus modificaciones ...,None,None,Entrega por Internet en la PNT,0019/04/22,Notificación lugar y fecha de entrega,Lugar para obtener la información:Calle Pachuc...,None,2019/04/22,México,Ciudad de México,BENITO JUAREZ,3020.0,Economía,None,NaN,None
495511,0001000038919,2019-02-25 12:06:24,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,PROCESO DE CREACIÓN EMISIÓN APROBACIÓN Y PUB...,FAVOR DE REVISAR ESCRITO ESPECIFICANDO LA INF...,https://www.infomex.org.mx/gobiernofederal/mod...,Copia Simple,0019/04/22,Notificación de envío,Número de Guía:EE932404261MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,XOCHIMILCO,16100.0,Economía,pdf,1.0,None
498021,0001000040919,2019-02-27 13:16:43,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,1.- Solicito copias simples en formato de ver...,NaN,None,Entrega por Internet en la PNT,0019/04/22,Notificación de envío,Número de Guía:EE932404275MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,COYOACAN,4310.0,Economía,pdf,1.0,None
498052,0001000041019,2019-02-27 13:23:52,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,1.- Solicito copias simples en formato de ver...,NaN,None,Entrega por Internet en la PNT,0019/04/22,Notificación de envío,Número de Guía:EE932404289MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,COYOACAN,4310.0,Economía,pdf,1.0,None


Al parecer hay casos en la columna "FECHALIMITE" que en lugar de venir el año como 2019 lo pusieron como 0019. Revisaremos si existen mas casos aparte de los que identificamos.

In [12]:
filtro = df['FECHALIMITE'].str.contains('0019/', na=False, regex=True)
aux3 = df[filtro]
len(aux3)

6

In [13]:
aux3

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
480809,0001000025519,2019-02-05 18:27:36,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,Proporcione el contrato -y sus modificaciones ...,None,None,Entrega por Internet en la PNT,0019/04/22,Notificación lugar y fecha de entrega,Lugar para obtener la información:Calle Pachuc...,None,2019/04/22,México,Ciudad de México,BENITO JUAREZ,3020.0,Economía,None,NaN,None
495511,0001000038919,2019-02-25 12:06:24,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,PROCESO DE CREACIÓN EMISIÓN APROBACIÓN Y PUB...,FAVOR DE REVISAR ESCRITO ESPECIFICANDO LA INF...,https://www.infomex.org.mx/gobiernofederal/mod...,Copia Simple,0019/04/22,Notificación de envío,Número de Guía:EE932404261MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,XOCHIMILCO,16100.0,Economía,pdf,1.0,None
498021,0001000040919,2019-02-27 13:16:43,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,1.- Solicito copias simples en formato de ver...,NaN,None,Entrega por Internet en la PNT,0019/04/22,Notificación de envío,Número de Guía:EE932404275MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,COYOACAN,4310.0,Economía,pdf,1.0,None
498052,0001000041019,2019-02-27 13:23:52,SECRETARÍA DE ECONOMÍA (SE),Terminada,Electrónica,Información Pública,1.- Solicito copias simples en formato de ver...,NaN,None,Entrega por Internet en la PNT,0019/04/22,Notificación de envío,Número de Guía:EE932404289MX<br>Fecha de Envío...,https://www.infomex.org.mx/gobiernofederal/mod...,2019/04/22,México,Ciudad de México,COYOACAN,4310.0,Economía,pdf,1.0,None
1356991,1857500107218,2018-10-24 19:32:51,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Con pago realizado,Electrónica,Información Pública,Licitación Pública No.18575021-524-12. Contrat...,Gerencia de Suministros y Servicios Administra...,https://www.infomex.org.mx/gobiernofederal/mod...,Entrega por Internet en la PNT,0019/01/15,Notificación de pago,El pago fue realizado con fecha 14/12/2018,None,2018/12/14,México,Veracruz,COATZACOALCOS,96530.0,Energía,None,NaN,None
1377012,1857500119118,2018-11-21 12:06:58,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Con pago realizado,Electrónica,Datos Personales,DESCRIPCIÓN SOLICITUD,OTROS DATOS,None,Copia Simple,0019/01/17,Notificación de pago,TEXTO RESPUESTA,None,2018/12/18,México,Tabasco,CENTRO,86090.0,Energía,None,NaN,None


Corregiremos estos 6 casos, imputaremos las fechas correctas manualmente.

In [14]:
df['FECHALIMITE'].loc[480809] = '2019/04/22'
df['FECHALIMITE'].loc[495511] = '2019/04/22'
df['FECHALIMITE'].loc[498021] = '2019/04/22'
df['FECHALIMITE'].loc[498052] = '2019/04/22'
df['FECHALIMITE'].loc[1356991] = '2019/01/15'
df['FECHALIMITE'].loc[1377012] = '2019/01/17'

/Users/diegovillalizarraga/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Ahora trataremos de corregir el tipo de la columna "FECHALIMITE" una vez mas.

In [15]:
df['FECHALIMITE'] = pd.to_datetime(df['FECHALIMITE'])

In [16]:
df['FECHARESPUESTA'] = pd.to_datetime(df['FECHARESPUESTA'])

In [17]:
df.head()

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,2012-03-02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
1,1857500002012,2012-01-01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,2012-03-02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
2,0001800000712,2012-01-01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012-02-02,Inexistencia de la información solicitada,EL ASUNTO SE TURNÓ A LA DIRECCIÓN GENERAL DE R...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012-02-02,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0,INIDOS SECRETARIA DE HACIENDA Y CREDITO PUBLI...
4,0001500002812,2012-01-01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012-02-02,Entrega de información en medio electrónico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0,None


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559012 entries, 0 to 1559011
Data columns (total 23 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   FOLIO                        1559012 non-null  object        
 1   FECHASOLICITUD               1559012 non-null  datetime64[ns]
 2   DEPENDENCIA                  1559012 non-null  object        
 3   ESTATUS                      1559012 non-null  object        
 4   MEDIOENTRADA                 1559012 non-null  object        
 5   TIPOSOLICITUD                1559012 non-null  object        
 6   DESCRIPCIONSOLICITUD         1559000 non-null  object        
 7   OTROSDATOS                   617311 non-null   object        
 8   ARCHIVOADJUNTOSOLICITUD      180445 non-null   object        
 9   MEDIOENTREGA                 1559011 non-null  object        
 10  FECHALIMITE                  1392031 non-null  datetime64[ns]
 11  RESPUESTA  

Ahora podemos ver que el tipo de todas las columnas es el correcto.

## Strings a lowercase

In [19]:
df = etl.StringLowercase(df)

In [20]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploración y producción,terminada,electrónica,información pública,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de información en medio electrónico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,méxico,distrito federal,benito juarez,3100,energía,pdf,1.0,pemex exploracion y produccion oficio fecha 0...
1,1857500002012,2012-01-01 05:13:16,pemex exploración y producción,terminada,electrónica,información pública,del atg para precisar información y datos usa...,None,None,entrega por internet en el infomex,2012-03-02,entrega de información en medio electrónico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,méxico,distrito federal,benito juarez,3100,energía,pdf,1.0,pemex exploracion y produccion oficio fecha 0...
2,0001800000712,2012-01-01 09:46:39,secretaría de energía,terminada,electrónica,información pública,agradeceré se me proporcione en archivo magnét...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la información solicitada,el asunto se turnó a la dirección general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,méxico,distrito federal,miguel hidalgo,11230,energía,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaría de hacienda y crédito público,terminada,electrónica,información pública,agradeceré se me proporcione en archivo magnét...,None,None,entrega por internet en el infomex,2012-02-02,la información está disponible públicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,méxico,distrito federal,miguel hidalgo,11230,hacienda y crédito público,pdf,1.0,inidos secretaria de hacienda y credito publi...
4,0001500002812,2012-01-01 10:32:15,secretaría de desarrollo agrario territorial ...,terminada,electrónica,información pública,agradeceré se me proporcione en archivo magnét...,None,None,entrega por internet en el infomex,2012-02-02,entrega de información en medio electrónico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,méxico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


Ahora podemos ver que todo el dataframe se encuentra en lowercase.

## Quitar acentos de Strings

Quitaremos acentos, dieresis y eñes.

In [21]:
df = etl.StringAcentos(df)

In [22]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 0...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usa...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 0...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito publi...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial ...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


## Quitar espacios al inicio y al final de los Strings

In [23]:
df = etl.StringStrip(df)

In [24]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usa...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial ...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


## Quitar espacios dobles (o mas) de los Strings.

In [25]:
df = etl.StringEspacios(df)

In [26]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usad...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


## Eliminamos observaciones o columnas con puros nulos

In [27]:
df = etl.EliminarNulos(df)

In [28]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usad...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


## Eliminamos observaciones repetidas

In [29]:
df = df.drop_duplicates()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559012 entries, 0 to 1559011
Data columns (total 23 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   folio                        1559012 non-null  object        
 1   fechasolicitud               1559012 non-null  datetime64[ns]
 2   dependencia                  1559012 non-null  object        
 3   estatus                      1559012 non-null  object        
 4   medioentrada                 1559012 non-null  object        
 5   tiposolicitud                1559012 non-null  object        
 6   descripcionsolicitud         1558999 non-null  object        
 7   otrosdatos                   617306 non-null   object        
 8   archivoadjuntosolicitud      180445 non-null   object        
 9   medioentrega                 1559011 non-null  object        
 10  fechalimite                  1392031 non-null  datetime64[ns]
 11  respuesta  

## Data Profiling

**Columnas Númericas**

In [31]:
df.describe()

,cantidad_archivos_respuesta
count,982794.000000
mean,1.817854
std,4.480219
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,796.000000


**Columnas String**

In [32]:
df.describe(include=[np.object])

,folio,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,respuesta,textorespuesta,archivorespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,texto_respuesta_adjunto
count,1559012,1559012,1559012,1559012,1559012,1558999,617306,180445,1559011,1559012,1457769,1329733,1559012,1300775,1300824,1559012,1557509,982794,223987
unique,1559012,1531,11,2,3,913588,202742,128233,8,19,348284,1049170,133,681,3100,31533,26,14,216431
top,0064101507615,instituto mexicano del seguro social,terminada,electronica,informacion publica,descripcion solicitud,otros datos,adjunto solicitud,entrega por internet en la pnt,entrega de informacion en medio electronico,texto respuesta,ADJUNTO,mexico,distrito federal,cuauhtemoc,nan,aportaciones a seguridad social,pdf,sr. vicente fox quezada: con respecto a sus so...
freq,1,160074,1355612,1496202,1249668,309344,309344,52213,706345,1026388,309344,280564,1550127,356554,114054,263005,317640,550781,474


**Columnas fecha**

In [33]:
df.describe(include=[np.datetime64])

,fechasolicitud,fechalimite,fecharespuesta
count,1559012,1392031,1559012
unique,1449746,1966,2593
top,2017-02-08 18:12:22,2019-08-26 00:00:00,2019-07-29 00:00:00
freq,10,8588,7014
first,2012-01-01 05:04:07,2012-01-19 00:00:00,2012-01-02 00:00:00
last,2019-12-31 22:16:29,2020-02-12 00:00:00,2020-01-13 00:00:00


### Review

In [34]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usad...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


In [35]:
df.tail()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
1559007,0064100022120,2019-12-31 21:09:17,instituto mexicano del seguro social (imss),en proceso,electronica,informacion publica,en relacion al oficio 129001051100/240 con fec...,delegacion imss acapulco de juarez,None,copia simple,2020-02-05,sin respuesta,None,None,2019-12-31,mexico,guerrero,acapulco de juarez,39713.0,aportaciones a seguridad social,None,NaN,None
1559008,0063500005020,2019-12-31 21:23:42,instituto del fondo nacional de la vivienda pa...,en proceso,electronica,informacion publica,siendo el dia de hoy 31 de diciembre de 2019 u...,oficio y autorizacion en el diario oficial de ...,None,entrega por internet en la pnt,2020-02-05,sin respuesta,None,None,2019-12-31,mexico,NaN,NaN,nan,ninguno,None,NaN,None
1559009,0064100022220,2019-12-31 21:28:41,instituto mexicano del seguro social (imss),en espera de ampliacion de informacion,electronica,datos personales,descripcion solicitud,otros datos,None,copia simple,NaT,requerimiento de informacion adicional,texto respuesta,ADJUNTO,2020-01-08,mexico,tamaulipas,san fernando,87625.0,aportaciones a seguridad social,None,NaN,None
1559010,0063500005120,2019-12-31 21:48:25,instituto del fondo nacional de la vivienda pa...,en proceso,electronica,datos personales,descripcion solicitud,otros datos,adjunto solicitud,copia simple,2020-02-05,sin respuesta,texto respuesta,None,2019-12-31,mexico,NaN,NaN,nan,ninguno,None,NaN,None
1559011,0064100022320,2019-12-31 22:16:29,instituto mexicano del seguro social (imss),en espera de ampliacion de informacion,electronica,datos personales,descripcion solicitud,otros datos,None,entrega por internet en la pnt,NaT,requerimiento de informacion adicional,texto respuesta,ADJUNTO,2020-01-08,mexico,tamaulipas,san fernando,87600.0,aportaciones a seguridad social,None,NaN,None


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559012 entries, 0 to 1559011
Data columns (total 23 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   folio                        1559012 non-null  object        
 1   fechasolicitud               1559012 non-null  datetime64[ns]
 2   dependencia                  1559012 non-null  object        
 3   estatus                      1559012 non-null  object        
 4   medioentrada                 1559012 non-null  object        
 5   tiposolicitud                1559012 non-null  object        
 6   descripcionsolicitud         1558999 non-null  object        
 7   otrosdatos                   617306 non-null   object        
 8   archivoadjuntosolicitud      180445 non-null   object        
 9   medioentrega                 1559011 non-null  object        
 10  fechalimite                  1392031 non-null  datetime64[ns]
 11  respuesta  

## Guardamos en formato parquet en S3 en la carpeta clean

Es importante guardar cada columna con el tipo correcto.

In [39]:
s3_url = f's3://inai-summerofdata/clean/inai.parquet'
formatos = {'folio':str, 'fechasolicitud':'datetime64', 'dependencia':str,  'estatus':str,
           'medioentrada':str, 'tiposolicitud':str, 'descripcionsolicitud':str, 'otrosdatos':str,
           'archivoadjuntosolicitud':str, 'medioentrega':str, 'fechalimite':'datetime64',
           'respuesta':str, 'textorespuesta':str, 'archivorespuesta':str, 'fecharespuesta':'datetime64',
           'pais':str, 'estado':str, 'municipio':str, 'codigopostal':str, 'sector':str,
           'tipo_archivo_respuesta':str, 'cantidad_archivos_respuesta':'float64', 
            'texto_respuesta_adjunto':str}
df.astype(formatos).to_parquet(s3_url, engine='auto', compression='snappy')